In [2]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
from jupyter_dash import JupyterDash
from datetime import datetime, date
import requests
load_figure_template("flatly")
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

In [2]:
locations_r = requests.get('https://api.ipma.pt/open-data/distrits-islands.json')

In [3]:
locations_dict = locations_r.json()

In [23]:
locations_dict.keys()

dict_keys(['owner', 'country', 'data'])

In [15]:
locals_df = pd.DataFrame(locations_dict['data'])
locals_df.info()
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   idRegiao       35 non-null     int64 
 1   idAreaAviso    35 non-null     object
 2   idConcelho     35 non-null     int64 
 3   globalIdLocal  35 non-null     int64 
 4   latitude       35 non-null     object
 5   idDistrito     35 non-null     int64 
 6   local          35 non-null     object
 7   longitude      35 non-null     object
dtypes: int64(4), object(4)
memory usage: 2.3+ KB


In [16]:
coordinates = ['latitude', 'longitude']
locals_df[coordinates] = locals_df[coordinates].astype('float64')
locals_df.head()

,idRegiao,idAreaAviso,idConcelho,globalIdLocal,latitude,idDistrito,local,longitude
0,1,AVR,5,1010500,40.6413,1,Aveiro,-8.6535
1,1,BJA,5,1020500,38.0200,2,Beja,-7.8700
2,1,BRG,3,1030300,41.5475,3,Braga,-8.4227
3,1,BRG,8,1030800,41.4434,3,Guimarães,-8.2938
4,1,BGC,2,1040200,41.8076,4,Bragança,-6.7606


In [18]:
global_local_id = list(locals_df['globalIdLocal'])

In [19]:

forecast_r = requests.get(f'https://api.ipma.pt/open-data/forecast/meteorology/cities/daily/{global_local_id[0]}.json')
forecast_dict = forecast_r.json()

In [24]:
forecast_dict.keys()

dict_keys(['owner', 'country', 'data', 'globalIdLocal', 'dataUpdate'])

In [28]:
pd.DataFrame(forecast_dict['data'])

,precipitaProb,tMin,tMax,predWindDir,idWeatherType,classWindSpeed,longitude,forecastDate,classPrecInt,latitude
0,100.0,14.4,16.4,S,9,2,-8.6535,2023-03-08,2.0,40.6413
1,100.0,14.1,17.6,SW,6,2,-8.6535,2023-03-09,2.0,40.6413
2,86.0,13.5,17.6,S,10,2,-8.6535,2023-03-10,1.0,40.6413
3,100.0,14.1,17.9,SW,9,1,-8.6535,2023-03-11,2.0,40.6413
4,27.0,13.2,18.4,S,3,2,-8.6535,2023-03-12,NaN,40.6413


In [6]:
id_stations_r = requests.get('https://api.ipma.pt/open-data/observation/meteorology/stations/stations.json')

In [7]:
id_stations_dict = id_stations_r.json()
id_stations_dict[0]

{'geometry': {'type': 'Point', 'coordinates': [-7.821, 37.033]},
 'type': 'Feature',
 'properties': {'idEstacao': 1210881, 'localEstacao': 'Olhão, EPPO'}}

In [8]:
id_stations_list = [station['properties'] for station in id_stations_dict]
id_stations_list[:5]

[{'idEstacao': 1210881, 'localEstacao': 'Olhão, EPPO'},
 {'idEstacao': 1210883, 'localEstacao': 'Tavira'},
 {'idEstacao': 11217430,
  'localEstacao': 'Graciosa / Serra das Fontes (DROTRH)'},
 {'idEstacao': 6212121, 'localEstacao': 'Terras de Bouro/Barral (CIM)'},
 {'idEstacao': 6212122, 'localEstacao': 'Amares Caldelas (CIM)'}]

In [9]:
idstations_df = pd.DataFrame(id_stations_list)
idstations_df.head()

,idEstacao,localEstacao
0,1210881,"Olhão, EPPO"
1,1210883,Tavira
2,11217430,Graciosa / Serra das Fontes (DROTRH)
3,6212121,Terras de Bouro/Barral (CIM)
4,6212122,Amares Caldelas (CIM)


In [10]:
coordinates_stations = [station['geometry']['coordinates'] for station in id_stations_dict]
coordinates_stations[:5]

[[-7.821, 37.033],
 [-7.62050375, 37.12166968],
 [-28.0038, 39.0672],
 [-8.31808611, 41.70225278],
 [-8.37977778, 41.66796389]]

In [11]:
idstations_df = pd.concat([idstations_df, pd.DataFrame(coordinates_stations)],
                          axis=1)
idstations_df.head()

,idEstacao,localEstacao,0,1
0,1210881,"Olhão, EPPO",-7.821000,37.033000
1,1210883,Tavira,-7.620504,37.121670
2,11217430,Graciosa / Serra das Fontes (DROTRH),-28.003800,39.067200
3,6212121,Terras de Bouro/Barral (CIM),-8.318086,41.702253
4,6212122,Amares Caldelas (CIM),-8.379778,41.667964


In [12]:
idstations_df.columns = ['id', 'local', 'lon', 'lat']
idstations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      197 non-null    int64  
 1   local   197 non-null    object 
 2   lon     197 non-null    float64
 3   lat     197 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 6.3+ KB


In [13]:
idstations_df.sort_values('local', inplace=True)
# idstations_df.to_csv('./data/idstations.csv')

In [11]:
idstations_df[idstations_df['local'].str.contains('Lisboa')]

,id,local,lon,lat
167,1200579,Lisboa (G.Coutinho),-9.127494,38.766203
135,1200535,Lisboa (Geofísico),-9.149722,38.719078
47,7240919,"Lisboa, Amoreiras (LFCL)",-9.164900,38.723900
86,1210762,"Lisboa, Tapada da Ajuda",-9.182825,38.709561


In [12]:
map = px.scatter_mapbox(idstations_df, lon='lon', lat='lat', 
                        center=dict(lon=-7.821000, lat=37.033000),
                        zoom=11, mapbox_style='carto-positron')
map.update_traces(marker_color='red', marker_size=8)
map.show()

In [3]:
obs_meteo = requests.get('https://api.ipma.pt/open-data/observation/meteorology/stations/observations.json')

In [4]:
obs_dict = obs_meteo.json()
obs_dict

{'2023-03-09T01:00': {'1210881': {'intensidadeVentoKM': 8.6,
   'temperatura': 16.5,
   'radiacao': -99.0,
   'idDireccVento': 6,
   'precAcumulada': 0.0,
   'intensidadeVento': 2.4,
   'humidade': 94.0,
   'pressao': 1017.7},
  '1210883': {'intensidadeVentoKM': 8.3,
   'temperatura': 15.9,
   'radiacao': -99.0,
   'idDireccVento': 6,
   'precAcumulada': 0.0,
   'intensidadeVento': 2.3,
   'humidade': 95.0,
   'pressao': 1018.6},
  '1210865': None,
  '11217430': {'intensidadeVentoKM': 18.7,
   'temperatura': 14.6,
   'radiacao': 0.0,
   'idDireccVento': 7,
   'precAcumulada': 0.0,
   'intensidadeVento': 5.2,
   'humidade': 99.0,
   'pressao': -99.0},
  '6212121': {'intensidadeVentoKM': 12.2,
   'temperatura': 12.6,
   'radiacao': 6.6,
   'idDireccVento': 6,
   'precAcumulada': 0.0,
   'intensidadeVento': 3.4,
   'humidade': 92.0,
   'pressao': 1009.0},
  '6212122': {'intensidadeVentoKM': 6.5,
   'temperatura': 13.9,
   'radiacao': 3.6,
   'idDireccVento': 5,
   'precAcumulada': 0.0,
  

In [5]:
obs_df = pd.DataFrame()
obs_df = pd.concat({k: pd.DataFrame(v).T for k, v in obs_dict.items()}, axis=0)
obs_df.reset_index(inplace=True)
obs_df['level_1'] = obs_df['level_1'].astype('int64')
obs_df.rename(columns={'level_0': 'date', 'level_1': 'id'}, inplace=True)
obs_df.head()

,date,id,intensidadeVentoKM,temperatura,radiacao,idDireccVento,precAcumulada,intensidadeVento,humidade,pressao
0,2023-03-09T01:00,1210881,8.6,16.5,-99.0,6.0,0.0,2.4,94.0,1017.7
1,2023-03-09T01:00,1210883,8.3,15.9,-99.0,6.0,0.0,2.3,95.0,1018.6
2,2023-03-09T01:00,1210865,None,None,None,None,None,None,None,None
3,2023-03-09T01:00,11217430,18.7,14.6,0.0,7.0,0.0,5.2,99.0,-99.0
4,2023-03-09T01:00,6212121,12.2,12.6,6.6,6.0,0.0,3.4,92.0,1009.0


In [14]:
obs_df = obs_df.merge(idstations_df, on='id')

In [18]:
# obs_df.head()
metrics = ['intensidadeVentoKM', 'temperatura', 'radiacao', 'idDireccVento', 
           'precAcumulada',	'intensidadeVento',	'humidade',	'pressao']
obs_df.dropna(subset=metrics)
obs_df.to_csv('./data/obs_outdated.csv')

In [18]:
obs_df['radiacao'].unique()

array([-99.0, None, 0.0, ..., 2876.3, 1685.5, 576.1], dtype=object)

In [19]:
obs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4728 entries, 0 to 4727
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   date                4728 non-null   object 
 1   id                  4728 non-null   int64  
 2   intensidadeVentoKM  4205 non-null   object 
 3   temperatura         4205 non-null   object 
 4   radiacao            4205 non-null   object 
 5   idDireccVento       4205 non-null   object 
 6   precAcumulada       4205 non-null   object 
 7   intensidadeVento    4205 non-null   object 
 8   humidade            4205 non-null   object 
 9   pressao             4205 non-null   object 
 10  local               4728 non-null   object 
 11  lon                 4728 non-null   float64
 12  lat                 4728 non-null   float64
dtypes: float64(2), int64(1), object(10)
memory usage: 517.1+ KB


In [20]:
obs_df['humidade'] = obs_df['humidade'].astype('float')

In [21]:
temp_df = obs_df[obs_df['id'] == 1210881]
temp = px.bar(temp_df, 'date', 'temperatura',
              color='humidade')

temp.show()

In [22]:
app = JupyterDash(
    __name__,
    external_stylesheets=[dbc.themes.FLATLY, dbc_css],
    meta_tags=[
        dict(
            name="viewport",
            content="width=device-width, initial-scale=1.0",
        ),
    ],
)

In [23]:
dropdown = dcc.Dropdown(
    id='dropdown',
    options=idstations_df['local'],
    value='Lisboa, Amoreiras (LFCL)'
)

graph = dcc.Graph(id='barplot')

app.layout = dbc.Container(id='container', class_name='dbc',
                           children=[
                               dbc.Col([
                                   dropdown,
                                   graph,
                               ])
                           ]
                           )

In [24]:
@app.callback(Output('barplot', 'figure'),
              [Input('dropdown', 'value')])
def location(value):
    df = obs_df[obs_df['local'] == value]
    fig = px.bar(df, 'date', 'temperatura', color='humidade')
    return fig

if __name__ == '__main__':
    app.run_server()

Dash app running on http://127.0.0.1:8050/


Dash app running on http://127.0.0.1:8050/


In [110]:
obs_df[obs_df['localEstacao'] == 'Amadora'].shape

(24, 12)